In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from osgeo import gdal

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pathlib import Path
import json
from random import randint
from collections import Counter
from collections import defaultdict

In [ ]:
from PIL import Image
from skimage import io
from matplotlib import pyplot as plt

In [ ]:
def get_disaster_dict(labels_generator):    
    disaster_dict = defaultdict(list)
    for label in labels_generator:
        disaster_type = label.name.split('_')[0]
        disaster_dict[disaster_type].append(str(label))
    return disaster_dict

In [ ]:
# Fetch all post disaster json files
labels_generator = Path('/content/drive/MyDrive/ML/geotiffs/tier1/labels').rglob(pattern=f'*post_*.json')

In [ ]:
disaster_dict = get_disaster_dict(labels_generator)

In [ ]:
disaster_dict.keys()

dict_keys(['hurricane-florence', 'hurricane-harvey', 'hurricane-matthew', 'socal-fire', 'mexico-earthquake', 'midwest-flooding', 'hurricane-michael', 'santa-rosa-wildfire', 'palu-tsunami', 'guatemala-volcano'])

In [ ]:
florence = np.array(disaster_dict['hurricane-florence'])

In [ ]:
harvey = np.array(disaster_dict['hurricane-harvey'])

In [ ]:
matthew = np.array(disaster_dict['hurricane-matthew'])

In [ ]:
michael = np.array(disaster_dict['hurricane-michael'])

In [ ]:
allTrainLabels = np.concatenate((florence, harvey, matthew, michael))

In [ ]:
# Fetch all post disaster json files
labels_generator = Path('/content/drive/MyDrive/ML/geotiffs/test/labels').rglob(pattern=f'*post_*.json')
disaster_dict = get_disaster_dict(labels_generator)
disaster_dict.keys()
florence = np.array(disaster_dict['hurricane-florence'])
harvey = np.array(disaster_dict['hurricane-harvey'])
matthew = np.array(disaster_dict['hurricane-matthew'])
michael = np.array(disaster_dict['hurricane-michael'])
allTestLabels = np.concatenate((florence, harvey, matthew, michael))

In [ ]:
# Fetch all post disaster json files
labels_generator = Path('/content/drive/MyDrive/ML/geotiffs/hold/labels').rglob(pattern=f'*post_*.json')
disaster_dict = get_disaster_dict(labels_generator)
disaster_dict.keys()
florence = np.array(disaster_dict['hurricane-florence'])
harvey = np.array(disaster_dict['hurricane-harvey'])
matthew = np.array(disaster_dict['hurricane-matthew'])
michael = np.array(disaster_dict['hurricane-michael'])
allHoldLabels = np.concatenate((florence, harvey, matthew, michael))

In [ ]:
len(allHoldLabels)

419

In [ ]:
len(allTestLabels)

421

In [ ]:
allTrainLabels

1262

In [ ]:
allTestImages = []
for elem in allTestLabels: 
  newElem = elem.replace('labels', 'images').replace('json', 'tif')
  allTestImages.append(newElem)

In [ ]:
allHoldImages = []
for elem in allHoldLabels: 
  newElem = elem.replace('labels', 'images').replace('json', 'tif')
  allHoldImages.append(newElem)

In [ ]:
len(allHoldImages)

419

In [ ]:
allTrainImages = []
for elem in allTrainLabels: 
  newElem = elem.replace('labels', 'images').replace('json', 'tif')
  allTrainImages.append(newElem)

In [ ]:
len(allTrainLabels)

1262

In [ ]:
allTrainImages

['/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-florence_00000023_post_disaster.tif',
 '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-florence_00000473_post_disaster.tif',
 '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-florence_00000390_post_disaster.tif',
 '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-florence_00000272_post_disaster.tif',
 '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-florence_00000066_post_disaster.tif',
 '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-florence_00000395_post_disaster.tif',
 '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-florence_00000455_post_disaster.tif',
 '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-florence_00000293_post_disaster.tif',
 '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-florence_00000193_post_disaster.tif',
 '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-florence_00000333_post_disaster.tif',
 '/content/drive/MyD

In [ ]:
import json


In [ ]:
f = open(allTrainLabels[0])

In [ ]:
data = json.load(f)

In [ ]:
data['features']['lng_lat'][0]['properties']['subtype']

'no-damage'

In [ ]:
idx = 0
while idx < len(allTrainLabels): 
  f = open(allTrainLabels[idx])
  data = json.load(f)
  if len(data['features']['lng_lat']) == 0: 
    allTrainLabels = np.delete(allTrainLabels, idx)
    allTrainImages = np.delete(allTrainImages, idx)
  else:
    idx += 1
  




In [ ]:
len(allTrainLabels)

1205

In [ ]:
len(allTrainImages)

1205

In [ ]:
idx = 0
while idx < len(allHoldLabels): 
  f = open(allHoldLabels[idx])
  data = json.load(f)
  if len(data['features']['lng_lat']) == 0: 
    allHoldLabels = np.delete(allHoldLabels, idx)
    allHoldImages = np.delete(allHoldImages, idx)
  else:
    idx += 1
  




In [ ]:
idx = 0
while idx < len(allTestLabels): 
  f = open(allTestLabels[idx])
  data = json.load(f)
  if len(data['features']['lng_lat']) == 0: 
    allTestLabels = np.delete(allTestLabels, idx)
    allTestImages = np.delete(allTestImages, idx)
  else:
    idx += 1
  




In [ ]:
idx = 0
yTrain = []
xTrain = []
while idx < len(allTrainLabels):
  f = open(allTrainLabels[idx])
  data = json.load(f)
  if data['features']['lng_lat'][0]['properties']['subtype'] != 'un-classified':
    yTrain.append(data['features']['lng_lat'][0]['properties']['subtype'])
    xTrain.append(allTrainImages[idx])
  idx += 1
yTrain = np.array(yTrain)
xTrain = np.array(xTrain)

In [ ]:
xTrain

array(['/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-florence_00000023_post_disaster.tif',
       '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-florence_00000473_post_disaster.tif',
       '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-florence_00000390_post_disaster.tif',
       ...,
       '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-michael_00000326_post_disaster.tif',
       '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-michael_00000208_post_disaster.tif',
       '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-michael_00000440_post_disaster.tif'],
      dtype='<U97')

In [ ]:
xTrain

array(['/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-florence_00000023_post_disaster.tif',
       '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-florence_00000473_post_disaster.tif',
       '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-florence_00000390_post_disaster.tif',
       ...,
       '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-michael_00000326_post_disaster.tif',
       '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-michael_00000208_post_disaster.tif',
       '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-michael_00000440_post_disaster.tif'],
      dtype='<U97')

In [ ]:
idx = 0
yHold = []
xHold = []
while idx < len(allHoldLabels):
  f = open(allHoldLabels[idx])
  data = json.load(f)
  if data['features']['lng_lat'][0]['properties']['subtype'] != 'un-classified':
    yHold.append(data['features']['lng_lat'][0]['properties']['subtype'])
    xHold.append(allHoldImages[idx])
  idx += 1
yHold = np.array(yHold)
xHold = np.array(xHold)

In [ ]:
idx = 0
yTest = []
xTest = []
while idx < len(allTestLabels):
  f = open(allTestLabels[idx])
  data = json.load(f)
  if data['features']['lng_lat'][0]['properties']['subtype'] != 'un-classified':
    yTest.append(data['features']['lng_lat'][0]['properties']['subtype'])
    xTest.append(allTestImages[idx])
  idx += 1
yTest = np.array(yTest)
xTest = np.array(xTest)

In [ ]:
newXTest = []
newYTest = []
count = 0
idx = 0
while idx < len(xTest): 
  elem = xTest[idx]
  try:
    img = io.imread(elem)
    img = np.array(img)
    data = img
    normalizedData = (data-np.min(data))/(np.max(data)-np.min(data))
    normalizedData = normalizedData*2
    normalizedData = normalizedData - 1
    newXTest.append(normalizedData)
    newYTest.append(yTest[idx])
  except OSError as e:
    count += 1
  idx += 1
  



newXTest = np.array(newXTest)
newYTest = np.array(newYTest)

In [ ]:
newXTrain = []
newYTrain = []
count = 0
idx = 0
while idx < len(xTrain): 
  elem = xTrain[idx]
  try:
    img = io.imread(elem)
    img = np.array(img)
    data = img
    normalizedData = (data-np.min(data))/(np.max(data)-np.min(data))
    normalizedData = normalizedData*2
    normalizedData = normalizedData - 1
    newXTrain.append(normalizedData)
    newYTrain.append(yTrain[idx])
  except OSError as e:
    count += 1
  idx += 1
  



newXTrain = np.array(newXTrain)
newYTrain = np.array(newYTrain)

In [ ]:
newXHold = []
newYHold = []
count = 0
idx = 0
while idx < len(xHold): 
  elem = xHold[idx]
  try:
    img = io.imread(elem)
    img = np.array(img)
    data = img
    normalizedData = (data-np.min(data))/(np.max(data)-np.min(data))
    normalizedData = normalizedData*2
    normalizedData = normalizedData - 1
    newXHold.append(normalizedData)
    newYHold.append(yHold[idx])
  except OSError as e:
    count += 1
  idx += 1
  



newXHold= np.array(newXHold)
newYHold = np.array(newYHold)

[2, 2]

In [ ]:
yTrain

array(['no-damage', 'major-damage', 'major-damage', ..., 'no-damage',
       'no-damage', 'no-damage'], dtype='<U12')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import shutil

In [ ]:
newXTrain.shape

(1110, 1024, 1024, 3)

In [ ]:
newYTrain.shape

(1110,)

In [ ]:
np.unique(newYTrain)

array(['destroyed', 'major-damage', 'minor-damage', 'no-damage'],
      dtype='<U12')

In [ ]:
import torch
from torch import nn

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0),
            nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0),
            nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0),
            nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0),
            nn.Flatten(),
            nn.Linear(128 * 9 * 9, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

model = MyModel()

In [ ]:
# Set the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.RMSprop(model.parameters(), lr=1e-4)

import os
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Data preprocessing and normalization
data_transforms = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])


In [ ]:


num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

train_acc_values = []
val_acc_values = []
train_loss_values = []
val_loss_values = []


In [ ]:
newYTrain[newYTrain == 'no-damage'] = 0

In [ ]:
newYTrain[newYTrain == 'minor-damage'] = 1
newYTrain[newYTrain == 'major-damage'] = 2
newYTrain[newYTrain == 'destroyed'] = 3

In [ ]:
newYTrain[newYTrain == '0'] = int(0)

In [ ]:
np.array([1,2,3,4])

array([1, 2, 3, 4])

In [ ]:
newXTrain

array(['0', '2', '2', ..., '0', '0', '0'], dtype='<U12')

In [ ]:
yTrain = []
for elem in newYTrain: 
  yTrain.append(int(elem))

yTrain = np.array(yTrain)
yTrain


array([0, 2, 2, ..., 0, 0, 0])

In [ ]:
train_data = []
xTrain
for i in range(len(newXTrain)):
   train_data.append([newXTrain[i], yTrain[i]])

In [ ]:
newYHold[newYHold == 'no-damage'] = 0
newYHold[newYHold == 'minor-damage'] = 1
newYHold[newYHold == 'major-damage'] = 2
newYHold[newYHold == 'destroyed'] = 3

In [ ]:
newYHold

array(['2', '2', '0', '0', '0', '0', '0', '2', '2', '0', '2', '0', '0',
       '0', '0', '0', '1', '0', '2', '0', '1', '0', '2', '2', '0', '2',
       '2', '2', '0', '0', '2', '0', '0', '2', '2', '2', '0', '0', '0',
       '2', '0', '0', '0', '2', '0', '0', '0', '1', '0', '0', '0', '2',
       '0', '0', '2', '0', '0', '0', '0', '0', '0', '0', '2', '2', '0',
       '2', '2', '0', '2', '2', '2', '2', '0', '0', '0', '0', '2', '0',
       '2', '0', '0', '2', '2', '0', '0', '0', '0', '2', '0', '2', '0',
       '3', '2', '2', '0', '2', '2', '0', '2', '2', '0', '2', '0', '0',
       '1', '0', '0', '0', '2', '0', '1', '2', '3', '0', '0', '0', '2',
       '2', '0', '0', '0', '2', '2', '1', '0', '0', '2', '0', '3', '2',
       '0', '2', '2', '0', '2', '0', '2', '0', '2', '0', '3', '0', '2',
       '2', '2', '0', '1', '2', '0', '0', '0', '1', '0', '2', '0', '0',
       '2', '2', '0', '0', '2', '1', '2', '2', '0', '0', '0', '0', '3',
       '2', '2', '1', '0', '3', '2', '0', '0', '1', '0', '2', '0

In [ ]:
yHold = []
for elem in newYHold: 
  yHold.append(int(elem))

yHold = np.array(yHold)
yHold


array([2, 2, 0, 0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 0, 0, 0, 1, 0, 2, 0, 1, 0,
       2, 2, 0, 2, 2, 2, 0, 0, 2, 0, 0, 2, 2, 2, 0, 0, 0, 2, 0, 0, 0, 2,
       0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2,
       2, 0, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 2, 0, 0, 0, 0, 2,
       0, 2, 0, 3, 2, 2, 0, 2, 2, 0, 2, 2, 0, 2, 0, 0, 1, 0, 0, 0, 2, 0,
       1, 2, 3, 0, 0, 0, 2, 2, 0, 0, 0, 2, 2, 1, 0, 0, 2, 0, 3, 2, 0, 2,
       2, 0, 2, 0, 2, 0, 2, 0, 3, 0, 2, 2, 2, 0, 1, 2, 0, 0, 0, 1, 0, 2,
       0, 0, 2, 2, 0, 0, 2, 1, 2, 2, 0, 0, 0, 0, 3, 2, 2, 1, 0, 3, 2, 0,
       0, 1, 0, 2, 0, 2, 1, 1, 0, 2, 3, 2, 2, 3, 0, 0, 1, 2, 1, 1, 3, 3,
       3, 1, 2, 0, 2, 1, 3, 2, 1, 2, 0, 3, 3, 1, 3, 2, 3, 3, 0, 1, 3, 3,
       2, 2, 1, 3, 2, 3, 1, 0, 1, 1, 3, 1, 3, 3, 2, 3, 3, 1, 1, 2, 3, 3,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 3, 1, 1, 0, 2, 2, 1, 2, 0, 1, 1, 0, 3,
       2, 3, 0, 2, 1, 2, 1, 1, 2, 0, 0, 0, 2, 0, 0, 1, 0, 0, 2, 0, 1, 0,
       2, 0, 1, 0, 0, 1, 1, 3, 1, 0, 1, 1, 2, 1, 1,

In [ ]:
val_data = []
for i in range(len(newXHold)):
   train_data.append([newXHold[i], yHold[i]])

In [ ]:

# Create data loaders
train_data   = torch.utils.data.TensorDataset(torch.tensor(newXTrain).to(device), torch.tensor(yTrain).to(device))
train_loader = DataLoader(train_data, batch_size=20, shuffle=True, num_workers=4)
validation_loader = DataLoader(val_data, batch_size=20, shuffle=False, num_workers=4)

# Training loop
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

train_acc_values = []
val_acc_values = []
train_loss_values = []
val_loss_values = []

In [ ]:
train_loader

In [ ]:
type(newXTrain[0][0][0][0])

numpy.float64

In [ ]:
newXTest

array([[[[-0.58893281, -0.43083004, -0.68379447],
         [-0.66007905, -0.53359684, -0.73913043],
         [-0.67588933, -0.54940711, -0.75494071],
         ...,
         [-0.63636364, -0.50988142, -0.66798419],
         [-0.66798419, -0.54940711, -0.69960474],
         [-0.6284585 , -0.50197628, -0.66007905]],

        [[-0.65217391, -0.5256917 , -0.7312253 ],
         [-0.63636364, -0.50988142, -0.72332016],
         [-0.60474308, -0.47035573, -0.69960474],
         ...,
         [-0.63636364, -0.50988142, -0.66798419],
         [-0.66798419, -0.54940711, -0.69960474],
         [-0.64426877, -0.51778656, -0.67588933]],

        [[-0.6916996 , -0.57312253, -0.76284585],
         [-0.63636364, -0.51778656, -0.7312253 ],
         [-0.59683794, -0.47826087, -0.69960474],
         ...,
         [-0.66007905, -0.54150198, -0.6916996 ],
         [-0.57312253, -0.43083004, -0.61264822],
         [-0.58102767, -0.43873518, -0.62055336]],

        ...,

        [[-0.6916996 , -0.47035573, -0

In [ ]:
newYTest[newYTest == 'no-damage'] = 0
newYTest[newYTest == 'minor-damage'] = 1
newYTest[newYTest == 'major-damage'] = 2
newYTest[newYTest == 'destroyed'] = 3

In [ ]:
allX = np.concatenate((newXTrain, newXHold, newXTest), axis=0)

In [ ]:
import torch

if torch.cuda.is_available():
    print("GPU is available!")
    device = torch.device("cuda")
else:
    print("GPU is not available. Using CPU.")
    device = torch.device("cpu")

GPU is available!


In [ ]:
import keras
from keras.utils import np_utils
from keras.models import Model, Sequential


In [ ]:
from keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.applications.xception import Xception
from keras.layers import Input, Dense, GlobalAveragePooling2D, Dense, LSTM, Dropout, GRU
from keras.preprocessing import sequence
from tensorflow.keras.optimizers import SGD, Adam
from keras import backend as K
from keras.layers import Input, Dense, GlobalAveragePooling2D, Dense, LSTM, Dropout, GRU

In [ ]:
from keras.layers import Embedding

In [ ]:
newXTrain.shape

(1110, 1024, 1024, 3)

In [ ]:
newXTrain = newXTrain.reshape(1110, 1024, 3072)
newXTrain.shape

(1110, 1024, 3072)

In [ ]:
yTrain

array(['no-damage', 'major-damage', 'major-damage', ..., 'no-damage',
       'no-damage', 'no-damage'], dtype='<U12')

In [ ]:
newXHold = newXHold.reshape(375, 1024, 3072)
newXHold.shape

(375, 1024, 3072)

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True,input_shape=(1024, 3072)))
# model.add(Dropout(0.4))
model.add(LSTM(64, return_sequences=True))
# model.add(Dropout(0.4))
model.add(LSTM(64)) 
# model.add(Dropout(0.4))
# model.add(Dense(5, activation='softmax'))
model.add(Dense(1, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(newXTrain, yTrain, validation_data=(newXHold, yHold), epochs=8, batch_size=256)
print(model.summary())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
saveModel = model

Epoch 1/8


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
##display accuracy of model for each play type
types = ['no-damage', 'minor-damage', 'major-damage', 'destroyed']
corrects =  [0.0 for _ in range(len(types))]
counts = [0.0 for _ in range(len(types))]

for idx in range(len(y_test)):
    counts[np.argmax(y_test[idx])] += 1.0
    pred = model.predict(X_test_pad[idx:idx+1])
    if np.argmax(y_test[idx]) == np.argmax(pred):
        corrects[np.argmax(y_test[idx])] += 1.0


for i in range(len(types)):
    print(types[i] + " Accuracy: %.2f%%" % (100 * corrects[i]/counts[i]))

In [ ]:
# yTrain

array([0, 2, 2, ..., 0, 0, 0])

In [ ]:
# newXTrain

array([[[[-0.71794872, -0.57264957, -0.8034188 ],
         [-0.69230769, -0.55555556, -0.78632479],
         [-0.69230769, -0.55555556, -0.78632479],
         ...,
         [-0.84615385, -0.7008547 , -0.87179487],
         [-0.83760684, -0.69230769, -0.86324786],
         [-0.79487179, -0.64102564, -0.82905983]],

        [[-0.76068376, -0.64102564, -0.83760684],
         [-0.75213675, -0.63247863, -0.82905983],
         [-0.70940171, -0.57264957, -0.79487179],
         ...,
         [-0.82051282, -0.68376068, -0.85470085],
         [-0.82905983, -0.69230769, -0.85470085],
         [-0.76923077, -0.61538462, -0.81196581]],

        [[-0.75213675, -0.63247863, -0.82905983],
         [-0.75213675, -0.63247863, -0.82905983],
         [-0.70940171, -0.57264957, -0.79487179],
         ...,
         [-0.8034188 , -0.67521368, -0.84615385],
         [-0.82051282, -0.69230769, -0.86324786],
         [-0.76923077, -0.63247863, -0.81196581]],

        ...,

        [[-0.75213675, -0.57264957, -0

In [ ]:
yTrain[yTrain == 1] = 5 

<ipython-input-43-21964494a8f4>:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  yTrain[yTrain == 1] = 5


In [ ]:
# yTrain

array([0, 2, 2, ..., 0, 0, 0])

In [ ]:
yTrain[yTrain == 0] = 1 #1 is no-damage

<ipython-input-44-f4896f3a8090>:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  yTrain[yTrain == 0] = 1 #1 is no-damage


In [ ]:
yTrain[yTrain == 2] = 0
yTrain[yTrain == 3] = 0
yTrain[yTrain == 5] = 0


<ipython-input-45-ff590d61f820>:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  yTrain[yTrain == 2] = 0
<ipython-input-45-ff590d61f820>:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  yTrain[yTrain == 3] = 0
<ipython-input-45-ff590d61f820>:3: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  yTrain[yTrain == 5] = 0


In [ ]:
yTrain.shape

(1152,)

In [ ]:
xTrain.shape

(1152,)

In [ ]:
tensorX = torch.from_numpy(newXTrain)
tensorY = torch.from_numpy(yTrain)

In [ ]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
data_transforms = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [ ]:
tensorX.shape

torch.Size([1110, 1024, 1024, 3])

In [ ]:
from PIL import Image as im

In [ ]:
import torch
from torch import nn

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0),
            nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0),
            nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0),
            nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0),
            nn.Flatten(),
            nn.Linear(128 * 9 * 9, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

model = MyModel()

In [ ]:
# Instantiate the model architecture
model_path = '/content/drive/MyDrive/ML/model_baseline2.pth'
loaded_model = MyModel()
# Load the saved state_dict
loaded_model.load_state_dict(torch.load(model_path))
# Set the model to evaluation mode
loaded_model.eval()

MyModel(
  (model): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU()
    (11): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (12): Flatten(start_dim=1, end_dim=-1)
    (13): Linear(in_features=10368, out_features=512, bias=True)
    (14): ReLU()
    (15): Linear(in_features=512, out_features=1, bias=True)
    (16): Sigmoid()
  )
)

In [ ]:
temp = tensorX.resize_(1110, 150, 150, 3)

In [ ]:
temp.shape

torch.Size([1110, 150, 150, 3])

In [ ]:
data_transforms = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [ ]:
test_dataset = datasets.ImageFolder(testing, transform=data_transforms)

In [ ]:
testing = '/content/drive/MyDrive/ML/geotiffs/tier1/testFold'

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=20, shuffle=False, num_workers=4)

In [ ]:
for inputs, labels in test_loader:
  print(inputs.shape)
  break

torch.Size([20, 3, 150, 150])


In [ ]:
if torch.cuda.is_available():
    print("GPU is available!")
    device = torch.device("cuda")
else:
    print("GPU is not available. Using CPU.")
    device = torch.device("cpu")

GPU is available!


In [ ]:
model.eval()
test_acc = 0.0

with torch.no_grad():
    for inputs, labels in test_loader:
        # inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs).squeeze()
        test_acc += ((outputs > 0.5).float() == labels.float()).float().sum().item()
test_acc /= len(test_dataset)

print(f'Test Acc: {test_acc:.4f}')

Test Acc: 0.5735


In [ ]:
##Runtime, Precision, F1 score, Accuracy, False Positives and Negatives 
##PULL 1 correctly classified example

In [ ]:
allTrainImages

['/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-florence_00000023_post_disaster.tif',
 '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-florence_00000473_post_disaster.tif',
 '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-florence_00000390_post_disaster.tif',
 '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-florence_00000272_post_disaster.tif',
 '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-florence_00000066_post_disaster.tif',
 '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-florence_00000395_post_disaster.tif',
 '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-florence_00000455_post_disaster.tif',
 '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-florence_00000293_post_disaster.tif',
 '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-florence_00000193_post_disaster.tif',
 '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-florence_00000333_post_disaster.tif',
 '/content/drive/MyD

In [ ]:
allTrainLabels

array(['/content/drive/MyDrive/ML/geotiffs/tier1/labels/hurricane-florence_00000023_post_disaster.json',
       '/content/drive/MyDrive/ML/geotiffs/tier1/labels/hurricane-florence_00000473_post_disaster.json',
       '/content/drive/MyDrive/ML/geotiffs/tier1/labels/hurricane-florence_00000390_post_disaster.json',
       ...,
       '/content/drive/MyDrive/ML/geotiffs/tier1/labels/hurricane-michael_00000326_post_disaster.json',
       '/content/drive/MyDrive/ML/geotiffs/tier1/labels/hurricane-michael_00000208_post_disaster.json',
       '/content/drive/MyDrive/ML/geotiffs/tier1/labels/hurricane-michael_00000440_post_disaster.json'],
      dtype='<U98')

In [ ]:
!pip install tifffile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from PIL import Image
import tifffile

def convert_tif_to_jpeg(tif_path, jpeg_path):

    image_arr = tifffile.imread(tif_path)
    image_arr = image_arr * 255
    image_arr = image_arr.astype(np.uint8)
    im = Image.fromarray(image_arr)
    im.save(jpeg_path)

# Example usage:
tif_path = '/content/drive/MyDrive/ML/geotiffs/tier1/images/socal-fire_00001369_post_disaster.tif'
jpeg_path = "/content/drive/MyDrive/ML/output.jpeg"
convert_tif_to_jpeg(tif_path, jpeg_path)

In [ ]:
from osgeo import gdal

options_list = [
    '-ot Byte',
    '-of JPEG',
    '-b 3',
    '-scale'
]

options_string = " ".join(options_list)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f9064645210> >

In [ ]:
gdal.Translate(
    '/content/drive/MyDrive/ML/geotiffs/tier1/testFold/damage/testingImg.png',
    '/content/drive/MyDrive/ML/geotiffs/tier1/testFold/damage/socal-fire_00001369_post_disaster.tif',
    options=options_string
)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f9061e91870> >

In [ ]:
xTrain

array(['/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-florence_00000023_post_disaster.tif',
       '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-florence_00000473_post_disaster.tif',
       '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-florence_00000390_post_disaster.tif',
       ...,
       '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-michael_00000326_post_disaster.tif',
       '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-michael_00000208_post_disaster.tif',
       '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-michael_00000440_post_disaster.tif'],
      dtype='<U97')

In [ ]:
yTrain

array(['no-damage', 'major-damage', 'major-damage', ..., 'no-damage',
       'no-damage', 'no-damage'], dtype='<U12')

In [ ]:
yTrain[yTrain == 'no-damage'] = int(1)

In [ ]:
yTrain

array(['1', 'major-damage', 'major-damage', ..., '1', '1', '1'],
      dtype='<U12')

In [ ]:
yTrain[yTrain == 'minor-damage'] = int(0)
yTrain[yTrain == 'major-damage'] = int(0)
yTrain[yTrain == 'destroyed'] = int(0)

In [ ]:
yTrainFinal = []
for elem in yTrain: 
  yTrainFinal.append(int(elem))

yTrainFinal = np.array(yTrainFinal)
yTrainFinal

array([1, 0, 0, ..., 1, 1, 1])

In [ ]:
xTrain

array(['/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-florence_00000023_post_disaster.tif',
       '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-florence_00000473_post_disaster.tif',
       '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-florence_00000390_post_disaster.tif',
       ...,
       '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-michael_00000326_post_disaster.tif',
       '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-michael_00000208_post_disaster.tif',
       '/content/drive/MyDrive/ML/geotiffs/tier1/images/hurricane-michael_00000440_post_disaster.tif'],
      dtype='<U97')

In [ ]:
idx = 0
ct = 0
while idx < len(xTrain):
  pathVar = None
  if yTrainFinal[idx] == 1:
    pathVar = 'no_damage'
  else:
    pathVar = 'damage'

  finPath =  '/content/drive/MyDrive/ML/geotiffs/test/testFold/' + pathVar + '/img' + str(ct) + '.png'
  idx += 1
  ct += 1
    
  try:
    gdal.Translate(
      finPath,
      xTrain[idx],
      options=options_string
    )
  except: 
        print("Failed files,", xTrain[idx]) 
  
  